In [6]:
### Qdrant
#whatever data is inserteted in vectordb need to be processed
# the sentence transfomer will give u model, which can turn raw data into embeddings

# !pip install sentence-transformers

# we can use qdrant to store the embeddings
# we can run qdrant search query against this data

# !pip install qdrant-client


In [23]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
# load the pretrained embegging model
model = SentenceTransformer('all-MiniLM-L6-v2')


In [24]:
documents = [
    {
        "name": "The Time Machine",
        "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]



embedding_sequence = model.encode(documents)

print(embedding_sequence)

[[-0.12326241  0.04113709  0.02166526 ...  0.05442156  0.00774782
  -0.04123154]
 [-0.04115283  0.04903881 -0.03679396 ...  0.01691867 -0.03791045
   0.03140537]
 [ 0.01995244  0.03645111 -0.01849128 ... -0.05695052  0.03222319
  -0.00683257]
 ...
 [-0.08405627 -0.04998705  0.08362971 ... -0.03071102  0.04202531
   0.00083449]
 [-0.02606715  0.04743873  0.02626558 ...  0.04933592 -0.05910193
   0.09368218]
 [ 0.01029425  0.03848584 -0.01669618 ...  0.05444737  0.01090582
   0.00802407]]


In [25]:
from qdrant_client import QdrantClient,models
qdrant = QdrantClient(":memory:")
# create collection
qdrant.create_collection(collection_name="books",
                         vectors_config=models.VectorParams(
                             size=model.get_sentence_embedding_dimension(),
                             distance=models.Distance.COSINE,
                         ))

True

In [27]:
# upload the collection
qdrant.upload_points(
    collection_name="books",
    points=[
        models.PointStruct(
            id=idx, vector=model.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(documents)
    ],
)

In [32]:
hits = qdrant.query_points(
    collection_name="books",
    query=model.encode("extinction of planet").tolist(),
     query_filter=models.Filter(
        must=[models.FieldCondition(key="year", range=models.Range(gte=2000))]
    ),
    limit=1,
).points

for hit in hits:
    print(hit.payload, "score:", hit.score)

{'name': 'The Three-Body Problem', 'description': 'Humans encounter an alien civilization that lives in a dying system.', 'author': 'Liu Cixin', 'year': 2008} score: 0.35236433318469995
